# Load-kb-data
Notebook responsible to load docs in AI Search with semantic search

<div style="text-align: center; line-height: 0; padding-top: 9px;">
  <img src="https://raw.githubusercontent.com/Foiac/customerSpecialist/refs/heads/main/customer-specialist/FlowDrawn.png" alt="Clean Data" style="width: 800px">
</div>

### Paths

In [47]:
from azure.core.credentials import AzureKeyCredential
from azure.search.documents.indexes import SearchIndexClient
from azure.search.documents.indexes.models import (
    SearchIndex, SimpleField, SearchableField, SearchFieldDataType
)
from azure.search.documents import SearchClient
import uuid
import os
from dotenv import load_dotenv

### Confs

In [48]:
load_dotenv()

SERVICE_NAME = os.getenv("AZURE_SEARCH_SERVICE_NAME")
INDEX_NAME = os.getenv("AZURE_SEARCH_INDEX_NAME")
API_KEY = os.getenv("AZURE_SEARCH_API_KEY")

endpoint = f"https://{SERVICE_NAME}.search.windows.net"

### Index Client

In [49]:
index_client = SearchIndexClient(endpoint=endpoint, credential=AzureKeyCredential(API_KEY))

### Create Index

In [50]:
fields = [
    SimpleField(name="id", type=SearchFieldDataType.String, key=True),
    SearchableField(name="content", type=SearchFieldDataType.String, analyzer_name="en.lucene"),
    SearchableField(name="titulo", type=SearchFieldDataType.String)
]

index = SearchIndex(name=INDEX_NAME, fields=fields)

if INDEX_NAME in [idx.name for idx in index_client.list_indexes()]:
    index_client.delete_index(INDEX_NAME)

index_client.create_index(index)

print(f"Índice '{INDEX_NAME}' criado com sucesso!")

Índice 'procedimentos-kb' criado com sucesso!


### Documents

In [51]:
search_client = SearchClient(endpoint=endpoint, index_name=INDEX_NAME, credential=AzureKeyCredential(api_key))

def carregar_documentos_de_txt(caminho_arquivo):
    docs = []
    with open(caminho_arquivo, 'r', encoding='utf-8') as f:
        conteudo = f.read()

    blocos = conteudo.strip().split('---')
    for bloco in blocos:
        linhas = bloco.strip().splitlines()
        if not linhas:
            continue

        doc = {"id": str(uuid.uuid4())}
        titulo = ''
        content = []

        for linha in linhas:
            if linha.startswith('titulo:'):
                titulo = linha.replace('titulo:', '').strip()
                doc["titulo"] = titulo
            elif linha.startswith('content:'):
                content.append(linha.replace('content:', '').strip())
            else:
                content.append(linha.strip())

        doc["content"] = "\n".join(content).strip()
        docs.append(doc)

    return docs

docs = carregar_documentos_de_txt("procedimentos.txt")
for doc in docs:
    print(doc)


{'id': '08432945-4174-4f20-85d4-f8785b6bbdb5', 'titulo': 'Resetar senha de colaborador', 'content': 'Este procedimento descreve como um colaborador pode solicitar a redefinição de sua própria senha.\n\nRestrições:\n- O colaborador deve ser o titular da conta.\n- O acesso à ferramenta de autosserviço de senhas deve estar habilitado.\n- Disponível de segunda a sexta.\n- Caso não consiga concluir sozinho, o suporte pode auxiliar em horário comercial (08:00–18:00).\n\nPasso a passo:\n1. Acesse o portal de autosserviço (senha.empresa.com).\n2. Clique em “Esqueci minha senha”.\n3. Preencha as informações solicitadas (CPF, e-mail ou matrícula).\n4. Escolha o método de verificação: SMS, e-mail alternativo ou token autenticador.\n5. Após validação, defina uma nova senha conforme as políticas da empresa.\n6. Aguarde até 15 minutos para propagação no ambiente.\n\nObservações:\n- Caso o portal não esteja acessível, abra um chamado no Service Desk.\n- Em nenhuma hipótese a senha deve ser fornecida 

### Upload Documents

In [52]:
result = search_client.upload_documents(documents=docs)

print("Documentos enviados com sucesso:")
for r in result:
    print(r)

Documentos enviados com sucesso:
{'additional_properties': {}, 'key': '08432945-4174-4f20-85d4-f8785b6bbdb5', 'error_message': None, 'succeeded': True, 'status_code': 201}


### Test

In [54]:
pergunta = "Posso resetar a senha de um gerente sem aprovação?"

resultados = search_client.search(search_text=pergunta, top=3)

print("Contexto encontrado:")
for doc in resultados:
    print("-", doc['content'])

Contexto encontrado:
- Este procedimento descreve como um colaborador pode solicitar a redefinição de sua própria senha.

Restrições:
- O colaborador deve ser o titular da conta.
- O acesso à ferramenta de autosserviço de senhas deve estar habilitado.
- Disponível de segunda a sexta.
- Caso não consiga concluir sozinho, o suporte pode auxiliar em horário comercial (08:00–18:00).

Passo a passo:
1. Acesse o portal de autosserviço (senha.empresa.com).
2. Clique em “Esqueci minha senha”.
3. Preencha as informações solicitadas (CPF, e-mail ou matrícula).
4. Escolha o método de verificação: SMS, e-mail alternativo ou token autenticador.
5. Após validação, defina uma nova senha conforme as políticas da empresa.
6. Aguarde até 15 minutos para propagação no ambiente.

Observações:
- Caso o portal não esteja acessível, abra um chamado no Service Desk.
- Em nenhuma hipótese a senha deve ser fornecida por terceiros.
